In [6]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from datetime import date

API_KEY = "0c2ac21044aa8d18cf7e999d70aa6f7f"
CITY = "London"

USER = "root"
PASSWORD = "21298632"
HOST = "127.0.0.1"
PORT = 3306
DB = "sales_weatherinfo_db"

MYSQL_CONN_STRING = f"mysql+pymysql://{USER}:{quote_plus(PASSWORD)}@{HOST}:{PORT}/{DB}"

def fetch_weather(api_key, city):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}
    response = requests.get(url, params=params, timeout=20)
    if response.status_code != 200:
        try:
            print("API error:", response.json())
        except Exception:
            print("API non-JSON response:", response.text)
        response.raise_for_status()
    data = response.json()
    return {
        "weather_date": date.today(),
        "city": city,
        "temp_c": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "description": data["weather"][0]["description"],
    }

def ensure_weather_table(engine):
    ddl = """
    CREATE TABLE IF NOT EXISTS weather (
        weather_date DATE NOT NULL,
        city VARCHAR(100) NOT NULL,
        temp_c DECIMAL(5,2),
        humidity INT,
        description VARCHAR(255)
    );
    """
    with engine.connect() as conn:
        conn.execute(text(ddl))
        conn.commit()

def store_weather_to_db(weather_data, connection_string):
    engine = create_engine(connection_string, future=True, pool_pre_ping=True)
    ensure_weather_table(engine)
    insert_sql = text("""
    INSERT INTO weather (weather_date, city, temp_c, humidity, description)
    VALUES (:weather_date, :city, :temp_c, :humidity, :description)
    """)
    with engine.connect() as conn:
        conn.execute(insert_sql, weather_data)
        conn.commit()
    engine.dispose()
    print(f"Weather data for {weather_data['city']} stored successfully in MySQL.")

def main():
    weather = fetch_weather(API_KEY, CITY)
    store_weather_to_db(weather, MYSQL_CONN_STRING)

if __name__ == "__main__":
    main()

Weather data for London stored successfully in MySQL.
